In [1]:
import os
import sys
sys.path.append(os.path.join(os.getcwd().replace("distilled_models", "")))

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
from utils.config import Config
from utils.parse_dataset import NetworkDataset, load_datasets
from utils.load_models import LSTM_Models
from utils.knowledge_distillation import KnowledgeDistillation

In [2]:
conf = Config()
load_models = LSTM_Models()
teacher_model = load_models.lstm_4
teacher_model.load()
student_model = load_models.light_lstm_1
kd = KnowledgeDistillation(
    teacher=teacher_model,
    student=student_model,
    device=conf.device,
    distillation="relation",
    rnn=True
)

Checkpoint loaded from /global/D1/homes/jorgetf/Network-Packet-ML-Model/large_models/checkpoint/lstm_4.pth!


In [3]:
X_train, y_train, X_val, y_val, X_test, y_test = load_datasets(conf.datasets, "lstm")

# create train, val and test dataloaders
train_dataset = NetworkDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, conf.batch_size, shuffle=True)

val_dataset = NetworkDataset(X_val, y_val)
val_loader = DataLoader(val_dataset, conf.batch_size, shuffle=True)

test_dataset = NetworkDataset(X_test, y_test)
test_loader = DataLoader(test_dataset, conf.batch_size)

In [4]:
data, labels = next(iter(train_loader))
print(data.shape, labels.shape)

torch.Size([512, 513, 1]) torch.Size([512])


In [5]:
acc, train_loss, val_loss = kd.train_kd(train_loader, val_loader, conf.epochs)

Epoch: 1/10, Accuracy: 73.80%, Train loss: 0.0075, Val loss: 0.6146
Epoch: 2/10, Accuracy: 80.38%, Train loss: 0.0036, Val loss: 0.3592
Epoch: 3/10, Accuracy: 85.76%, Train loss: 0.0025, Val loss: 0.2429
Epoch: 4/10, Accuracy: 88.94%, Train loss: 0.0021, Val loss: 0.1952
Epoch: 5/10, Accuracy: 90.49%, Train loss: 0.0018, Val loss: 0.1710
Epoch: 6/10, Accuracy: 91.11%, Train loss: 0.0017, Val loss: 0.1552
Epoch: 7/10, Accuracy: 91.30%, Train loss: 0.0016, Val loss: 0.1447
Epoch: 8/10, Accuracy: 91.89%, Train loss: 0.0015, Val loss: 0.1396
Epoch: 9/10, Accuracy: 92.10%, Train loss: 0.0014, Val loss: 0.1335
Epoch: 10/10, Accuracy: 92.24%, Train loss: 0.0014, Val loss: 0.1296


In [6]:
# test teacher accuracy
loss, accuracy = teacher_model.evaluate(test_loader)
print(f"Teacher accuracy: {100*accuracy:.2f}%")

Teacher accuracy: 96.33%


In [7]:
# test student accuracy
loss, accuracy = student_model.evaluate(test_loader)
print(f"Student accuracy: {100*accuracy:.2f}%")

Student accuracy: 92.17%


In [8]:
student_model.save()

Checkpoint saved at /global/D1/homes/jorgetf/Network-Packet-ML-Model/distilled_models/checkpoint/lstm_1.pth
